In [1]:
import numpy as np
import pandas as pd
import json

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, CuDNNGRU
from tensorflow.python.keras.optimizers import adam_v2
import tensorflow as tf
import matplotlib.pyplot as plt

2023-01-09 19:20:36.437024: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def plot_graph_loss(history, metric, transparent=False):
    plt.plot(history.history[metric])
    plt.plot(history.history["val_" + metric], "")
    plt.xlabel("Devirler")
    plt.ylabel("Kayıp")
    plt.legend(["Kayıp", "Devir Test Kaybı"])
    plt.savefig(
        "eticaret" + metric + ".png",
        dpi=(250),
        bbox_inches="tight",
        transparent=transparent,
    )

    plt.close()

In [3]:
def plot_graph_accuracy(history, metric, transparent=False):
    plt.plot(history.history[metric])
    plt.plot(history.history["val_" + metric], "")
    plt.xlabel("Devirler")
    plt.ylabel("Doğruluk")
    plt.legend(["Doğruluk", "Devir Test Doğruluğu"])
    plt.savefig(
        "eticaret" + metric + ".png",
        dpi=(250),
        bbox_inches="tight",
        transparent=transparent,
    )

    plt.close()

In [6]:
yorumlar = pd.read_csv("yorumlar.csv")
target = yorumlar["Puan"].values.tolist()
data = yorumlar["Yorum"].values.tolist()

cutoff = int(len(data) * 0.90)
x_train, x_test = data[:cutoff], data[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]

In [3]:
with open("./third_tokenizer.json") as json_dosyasi:
    json_tokenizer = json.load(json_dosyasi)

In [37]:
json_tokenizer['larla']

19850

In [8]:
with open('tf_tokenizer.json') as f:
    json_string = json.load(f)

tf_tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(json_string)

In [39]:
tf_tokenizer.word_index['tarla']

177704

In [4]:
def tokenlestir(yorumListesi):
    y_yorumlar = []
    for yorum in yorumListesi:
        y_yorum = []
        for kelime in str(yorum).lower().split():
            if len(y_yorum) < 50 and kelime in json_tokenizer:
                y_yorum.append(json_tokenizer[kelime])

        if len(y_yorum) < 50:
            sifirlar = list(np.zeros(50 - len(y_yorum), dtype=int))
            y_yorum = sifirlar + y_yorum

        y_yorumlar.append(y_yorum)
    return np.array(y_yorumlar, dtype=np.dtype(np.int32))

In [10]:
egitim_kume = tokenlestir(x_train)
test_kume = tokenlestir(x_test)

In [16]:
model = Sequential()

model.add(
    Embedding(input_dim=20001, output_dim=50, input_length=50, name="embedding_layer")
)

model.add(GRU(units=16, return_sequences=True, reset_after=False))
model.add(GRU(units=8, return_sequences=True, reset_after=False))
model.add(GRU(units=4, return_sequences=False, reset_after=False))
model.add(Dense(1, activation="sigmoid"))

opt = adam_v2.Adam(learning_rate=1e-3, clipnorm=1.0, clipvalue=0.5)

model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 50, 50)            1000050   
_________________________________________________________________
gru_12 (GRU)                 (None, 50, 16)            3216      
_________________________________________________________________
gru_13 (GRU)                 (None, 50, 8)             600       
_________________________________________________________________
gru_14 (GRU)                 (None, 4)                 156       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 5         
Total params: 1,004,027
Trainable params: 1,004,027
Non-trainable params: 0
_________________________________________________________________


In [29]:
n_y_train = np.array(y_train)
n_y_test = np.array(y_test)

history = model.fit(
    egitim_kume,
    n_y_train,
    epochs=25,
    batch_size=512,
    validation_data=(test_kume, n_y_test),
    validation_steps=30,
)

Epoch 1/25
140/140 [==============================] - ETA: 0s - loss: 0.5299 - accuracy: 0.7679WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 30 batches). You may need to use the repeat() function when building your dataset.


140/140 [==============================] - 18s 88ms/step - loss: 0.5299 - accuracy: 0.7679 - val_loss: 0.3937 - val_accuracy: 0.8610
Epoch 2/25
140/140 [==============================] - 11s 81ms/step - loss: 0.3448 - accuracy: 0.8839
Epoch 3/25
140/140 [==============================] - 11s 81ms/step - loss: 0.2956 - accuracy: 0.9037
Epoch 4/25
140/140 [==============================] - 11s 79ms/step - loss: 0.2631 - accuracy: 0.9177
Epoch 5/25
140/140 [==============================] - 11s 78ms/step - loss: 0.2419 - accuracy: 0.9258
Epoch 6/25
140/140 [==============================] - 11s 79ms/step - loss: 0.2255 - accuracy: 0.9330
Epoch 7/25
140/140 [==============================] - 11s 78ms/step - loss: 0.2124 - accuracy: 0.9373
Epoch 8/25
140/140 [==============================] - 11s 78ms/step - loss: 0.1988 - accuracy: 0.9431
Epoch 9/25
140/140 [==============================] - 11s 80ms/step - loss: 0.1891 - accuracy: 0.9469
Epoch 10/25
140/140 [==============================

In [30]:
test_loss, test_acc = model.evaluate(test_kume, n_y_test)

249/249 [==============================] - 3s 12ms/step - loss: 0.4711 - accuracy: 0.8697


In [31]:
print("Test Loss: {}".format(test_loss))
print("Test Accuracy: {}".format(test_acc))

text1 = "bu ürün çok iyi herkese tavsiye ederim"
text2 = "kargo çok hızlı aynı gün elime geçti"
text3 = "büyük bir hayal kırıklığı yaşadım bu ürün bu markaya yakışmamış"
text4 = "mükemmel"
text5 = "tasarımı harika ancak kargo çok geç geldi ve ürün açılmıştı tavsiye etmem"
text6 = "hiç resimde gösterildiği gibi değil"
text7 = "kötü yorumlar gözümü korkutmuştu ancak hiçbir sorun yaşamadım teşekkürler"
text8 = "hiç bu kadar kötü bir satıcıya denk gelmemiştim ürünü geri iade ediyorum"
text9 = "tam bir fiyat performans ürünü"
text10 = "beklediğim gibi çıkmadı"
yazilar = [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10]

deneme_kume = tokenlestir(yazilar)
sonuc = model.predict(deneme_kume)
print(sonuc)

Test Loss: 0.47108957171440125
Test Accuracy: 0.8696581125259399
[[0.9590216 ]
 [0.95821506]
 [0.01047284]
 [0.95859253]
 [0.01047935]
 [0.220978  ]
 [0.95817643]
 [0.01048765]
 [0.9576007 ]
 [0.18795137]]


In [32]:
plot_graph_accuracy(history, "accuracy")

In [33]:
plot_graph_loss(history, "loss")

In [34]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model/assets


INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [2]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_layer (Embedding)  (None, 50, 50)           1000050   
                                                                 
 gru_12 (GRU)                (None, 50, 16)            3216      
                                                                 
 gru_13 (GRU)                (None, 50, 8)             600       
                                                                 
 gru_14 (GRU)                (None, 4)                 156       
                                                                 
 dense_4 (Dense)             (None, 1)                 5         
                                                                 
Total params: 1,004,027
Trainable params: 1,004,027
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

y_pred = new_model.predict(test_kume)
y_pred = (y_pred > 0.5)

cm = confusion_matrix(n_y_test, y_pred)
print(cm)

In [9]:
text1 = "ürün kaliteli ama kargo için yanı şeyi söyleyemem"
text2 = "kargo zamanında elime ulaştı ancak içi fiyasko"
text3 = "her şey istediğim gibi geldi, zaten fiyatı da öyleydi"
text4 = "aldığım 10 ürünün 8 tanesi istediğim gibiydi"
text5 = "kargo tam istediğim zamanında getirdi"
text6 = "en büyük kargo benim kargom"
text7 = "kötü yorumlar gözümü korkutmuştu ancak hiçbir sorun yaşamadım teşekkürler"
text8 = "hiç bu kadar kötü bir satıcıya denk gelmemiştim ürünü geri iade ediyorum"
text9 = "tam bir fiyat performans ürünü"
text10 = "beklediğim gibi çıkmadı"
yazilar = [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10]

deneme_kume = tokenlestir(yazilar)
sonuc = new_model.predict(deneme_kume)
print(sonuc)

1/1 [==============================] - 0s 36ms/step
[[0.01070459]
 [0.01220471]
 [0.95957804]
 [0.95780295]
 [0.95676   ]
 [0.8082179 ]
 [0.95817643]
 [0.01048765]
 [0.9576007 ]
 [0.18795137]]
